In [1]:
# %pip install pandas
# %pip install tensorflow[and-cuda]
%load_ext autoreload
%autoreload 2

In [2]:
# count = 0
# c2 = 0
# punctuations = ['.', ',', '?', '!']
# tokens = [ "[FULLSTOP]", "[COMMA]", "[QUESTIONMARK]", "[EXCLAMATIONMARK]" ]
# with open( "eng_sentences.tsv" ) as file:
#     with open( "eng_out.txt", "w" ) as w_file:
#         for line in file.readlines():
#             out = "[START] "
#             sentence = line.split("\t")[2][:-1]
#             words = sentence.split(" ")
#             # print(words)
#             for word in words:
#                 # print(word[-1])
#                 if( word[-1] in punctuations ):
#                     count = count +1 
#                     out = out + tokens[punctuations.index(word[-1])] + " "
#                 else:
#                     out = out + "[SPACE] "
#             out = out + "[END]"
#             out = out + "\n"
#             c2 = c2 +1
#             print(c2)
#             w_file.write(out)
            

In [3]:
from Utils import Dataset
from Transformer import Transformer
import tensorflow as tf
from tensorflow.keras.losses import SparseCategoricalCrossentropy
# from focal_loss import SparseCategoricalFocalLoss
INP_MAX_LENGTH = 50
OUT_MAX_LENGTH = 50
BUFFER_SIZE = 10000
BATCH_SIZE = 64

2024-08-16 23:09:50.582747: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-16 23:09:50.666624: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-16 23:09:50.688295: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-16 23:09:50.760465: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-16 23:09:52.028682: W tensorflow/compiler/tf2

In [4]:

with tf.device('/CPU:0'):
    ds = Dataset("eng_sentences.tsv", "eng_out.txt", INP_MAX_LENGTH, OUT_MAX_LENGTH, BATCH_SIZE, BUFFER_SIZE)


I0000 00:00:1723829993.004357  713178 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1723829993.222072  713178 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1723829993.222168  713178 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1723829993.227200  713178 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1723829993.227295  713178 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

In [5]:
train_batches, test_batches = ds.get_data_generators()

In [6]:
for (s,t), t_in  in train_batches.take(1):
    break

ds.decode_input(s[0])

'It was kind of scary                                             '

In [7]:
ds.decode_output(t_in[0])

'[SPACE] [SPACE] [SPACE] [SPACE] [FULLSTOP] [END]                                           '

In [8]:
ds.decode_output(t[0])

'[START] [SPACE] [SPACE] [SPACE] [SPACE] [FULLSTOP] [END]                                          '

In [9]:
# import os
# os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "TRUE"

In [10]:
ds.get_output_vocab_size()

9

In [11]:
import tensorflow as tf

In [12]:
import tensorflow.keras.backend as K
a = tf.keras.metrics.Accuracy()

In [13]:
def sparse_crossentropy_masked(y_true, y_pred):
    y_true_masked = tf.boolean_mask(y_true, tf.not_equal(y_true, 0))
    y_pred_masked = tf.boolean_mask(y_pred, tf.not_equal(y_true, 0))
    return K.mean(K.sparse_categorical_crossentropy(y_true_masked, y_pred_masked))

def masked_accuracy(y_true, y_pred):
    a.reset_state()
    # Create a mask for non-padded tokens
    mask = tf.math.logical_not(tf.math.equal(y_true, 0))
    y_pred = tf.argmax(y_pred, axis = 1)
    # Apply the mask to both predictions and true labels
    masked_y_true = tf.boolean_mask(y_true, mask)
    masked_y_pred = tf.boolean_mask(y_pred, mask)
    a.update_state( masked_y_true, masked_y_pred )
    # Calculate accuracy based on non-padded tokens
    return a.result()

In [ ]:
with tf.device("/GPU:0"):
    num_layers = 2
    d_model = 128
    dense_hidden_num_units = 64
    num_heads = 6

    transformer = Transformer(num_layers, d_model, num_heads, dense_hidden_num_units, ds.get_input_vocab_size(), ds.get_output_vocab_size(), INP_MAX_LENGTH, OUT_MAX_LENGTH)

    checkpoint_filepath = "./tmp/model/cp-{epoch:04d}.weights.h5"
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_filepath,
        save_weights_only=True,
        monitor='val_accuracy',
        mode='max',
        save_freq="epoch",
        save_best_only=True)

    callback_list = [ model_checkpoint_callback ]
    
    transformer.compile(
        optimizer="rmsprop",
        loss=SparseCategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"]
    )

    history = transformer.fit(
        train_batches,
        callbacks = callback_list ,
        epochs=20,
        validation_data=test_batches
    )

In [ ]:
class NamedEntityRecognizer(tf.Module):
  def __init__(self, transformer):
    self.transformer = transformer

  def __call__(self, sentence, max_length=OUT_MAX_LENGTH):
    sentence = ds.vectorize(sentence)
    # print(sentence)
    encoder_input = sentence

    start_end = ds.encode_label(["[START]","[END]"])
    start = start_end[0][0][tf.newaxis]
    end = start_end[1][0][tf.newaxis]
    # print(start)

    output_array = tf.TensorArray(dtype=tf.int64, size=0, dynamic_size=True)
    output_array = output_array.write(0, start)

    for i in tf.range(max_length):
      output = tf.transpose(output_array.stack())
      predictions = self.transformer([encoder_input, output], training=False)
      
      predictions = predictions[:, -1:, :]  
      predicted_id = tf.argmax(predictions, axis=-1)

      output_array = output_array.write(i+1, predicted_id[0])


    output = tf.transpose(output_array.stack())
    text = [ds.decode_output(out) for out in output]

    return output


In [ ]:
translator = NamedEntityRecognizer(transformer)


In [ ]:
for (s,t), t_l in train_batches.take(1):
    break

sentence = [ds.decode_input(s[4])]
print(sentence)

translated_text = translator(
    sentence
)
translated_text

['Grandmother looks very comfortable in that chair beside the fire                                                                                                                                                                                              ']


/home/shriyam/ASR/Punctuate/.venv/lib/python3.10/site-packages/keras/src/layers/layer.py:934: UserWarning: Layer 'query' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/home/shriyam/ASR/Punctuate/.venv/lib/python3.10/site-packages/keras/src/layers/layer.py:934: UserWarning: Layer 'key' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/home/shriyam/ASR/Punctuate/.venv/lib/python3.10/site-packages/keras/src/layers/layer.py:934: UserWarning: Layer 'value' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see 

<tf.Tensor: shape=(1, 51), dtype=int64, numpy=
array([[3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 5, 4, 0, 2, 5, 4, 0, 2, 5, 4, 0, 2,
        5, 4, 0, 2, 2, 2, 2, 2, 2, 5, 4, 0, 2, 5, 4, 0, 2, 5, 4, 0, 2, 2,
        5, 4, 0, 2, 2, 2, 2]])>

<tf.Tensor: shape=(49,), dtype=int64, numpy=
array([2, 2, 2, 2, 2, 2, 2, 2, 2, 5, 4, 0, 2, 5, 4, 0, 2, 5, 4, 0, 2, 5,
       4, 0, 2, 2, 2, 2, 2, 2, 5, 4, 0, 2, 5, 4, 0, 2, 5, 4, 0, 2, 2, 5,
       4, 0, 2, 2, 2])>

In [ ]:
t_l[4]

<tf.Tensor: shape=(49,), dtype=int64, numpy=
array([2, 2, 2, 2, 2, 2, 2, 2, 2, 5, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0])>

In [ ]:
t_l[4]

<tf.Tensor: shape=(49,), dtype=int64, numpy=
array([2, 2, 2, 2, 2, 2, 2, 2, 2, 5, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0])>

In [ ]:
ds.output_vectorizer.get_vocabulary()

['',
 '[UNK]',
 '[SPACE]',
 '[START]',
 '[END]',
 '[FULLSTOP]',
 '[QUESTIONMARK]',
 '[COMMA]',
 '[EXCLAMATIONMARK]']